### Extract embedding axis

In [ ]:
# !pip install --upgrade accelerate transformers

In [2]:
import pandas as pd

In [3]:
# Open training_embeddings.csv and load it into a dataframe
df = pd.read_csv('training_embeddings.csv')
df.head()

,word,0,1,2,3,4,5,6,7,8,...,758,759,760,761,762,763,764,765,766,767
0,companion,0.128,-0.082,0.109,-0.012,0.317,0.262,-0.016,0.128,0.357,...,0.395,-0.107,-0.026,-0.061,0.358,-0.138,-0.067,0.236,0.385,-0.115
1,toast,0.209,0.411,0.026,-0.000,-0.243,-0.325,0.148,0.238,-0.120,...,0.373,-0.174,0.119,-0.036,0.445,-0.029,0.145,0.169,0.364,-0.097
2,lounge,0.759,-0.116,0.116,0.113,0.460,-0.212,0.115,-0.032,0.137,...,0.259,0.070,-0.059,-0.114,0.441,-0.096,-0.074,0.196,0.215,-0.254
3,watch,0.401,-0.003,-0.061,-0.406,0.933,-0.140,-0.186,0.286,-0.170,...,0.459,-0.067,-0.266,-0.318,0.173,-0.109,0.219,-0.010,0.404,-0.161
4,haul,0.580,0.031,0.311,0.048,-0.102,0.081,0.047,0.423,-0.187,...,0.505,0.096,0.095,-0.009,0.453,0.003,0.337,0.259,0.064,0.092


In [19]:
def get_x_embedding(df, x):
  """
  Extract the x column from the dataframe and store it as a list
  """
  # Extract the word and x column from the dataframe and store them as a list of lists
  # index = 0
  words = df['word'].tolist()
  emd_index = df[str(x)].tolist()
  # data = [[words[i], emd_index[i]] for i in range(len(words))]
  # print(data[:5])

  # Concatenate the two lists into a list of lists
  data = [words,emd_index]
  return data

data = get_x_embedding(df, 0)
print(len(data), len(data[0]), len(data[1]))
print(data[0][:5])
print(data[1][:5])

2 383 383
['companion', 'toast', 'lounge', 'watch', 'haul']
[0.128, 0.209, 0.759, 0.401, 0.58]


### Get axis interpretation from OpenAI

In [ ]:
# install OpenAI api
# !pip install --upgrade openai

In [21]:
# load api key from secrets.json
import openai
import json

try:
    with open("secrets.json") as f:
        secrets = json.load(f)
    my_api_key = secrets["openai"]
    print("API key loaded.")
    openai.api_key = my_api_key
except FileNotFoundError:
    print("Secrets file not found. YOU NEED THEM TO RUN THIS.")

API key loaded.


In [24]:
axis_emb = get_x_embedding(df, 0)

In [32]:
# model_num = "gpt-3.5-turbo-1106"
# model_num = "gpt-4-1106-preview"
model_num = "gpt-4"

completion = openai.ChatCompletion.create(
  model=model_num,
  messages=[
    {"role": "system", "content": "You are an expert transformer embeddings labeller."},
    {"role": "user", "content": f"Below are two lists. The first list contains words that have been put into DistilBERT. DistilBERT creates an embedding with 768 dimensions or axes. The second list contains the embedding value from DistilBERT for one axis across the words. By carefully comparing and considering the embedding values for each word, please interpret the likely linguistic feature that this embedding axis encodes. This interpretation must be consistent across all the words and correspond to their respective positive, zero, or negative embedding values.\n\n  The output must be a Python dictionary with three items, the key is a string containing the possible interpretation of the axis. The value is a float, representing the confidence score from 0 to 1. \n\n Here is an output sample: {{<first interpretation>:<first interpretation confidence score>, <second interpretation>:<second interpretation confidence score>, <third interpretation>:<third interpretation confidence score>}} \n\n {axis_emb[0]}\n\n {axis_emb[1]}"}
  ]
)

print(completion.choices[0].message)

# log the stringified output into a txt file by appending it to the end of the file
with open("output.txt", "a") as f:
  f.write(str(completion))

{
  "role": "assistant",
  "content": "{\"Verb or Action Related\": 0.7, \"Instrument or Tools Associated\": 0.6, \"Emotion or Feeling Connotated\": 0.5}"
}


In [57]:
# convert the output string into a dictionary
interp_dict = eval(completion.choices[0].message.content)
print(interp_dict)

# convert the dictionary keys into a list
interp_keys = list(interp_dict.keys())
print(interp_keys[0])

{'Verb or Action Related': 0.7, 'Instrument or Tools Associated': 0.6, 'Emotion or Feeling Connotated': 0.5}
Verb or Action Related


### Evaluate interpretations using OpenAI

In [69]:
# test the prompt using smaller set
trunc_word_list = axis_emb[0][:50]
print(trunc_word_list)

['companion', 'toast', 'lounge', 'watch', 'haul', 'combination', 'majesty', 'extinguish', 'nutrient', 'bark', 'rest', 'hold', 'drainage', 'chill', 'time', 'drink', 'eat', 'journey', 'surf', 'shipment', 'measure', 'devour', 'cutlery', 'catch', 'drive', 'cooling', 'sail', 'flutter', 'moo', 'descend', 'commute', 'score', 'indulgence', 'cutting', 'containment', 'communication', 'transportation', 'glide', 'slice', 'drying', 'tote', 'nurture', 'breakfast', 'cut', 'throw', 'brush', 'competition', 'signal', 'carry', 'mobility']


In [70]:
# model_num = "gpt-3.5-turbo-1106"
# model_num = "gpt-4-1106-preview"
model_num = "gpt-4"

completion = openai.ChatCompletion.create(
  model=model_num,
  messages=[
    {"role": "system", "content": "You are an expert word sense scorer."},
    {"role": "user", "content": f"For the list of words below, please assign it a score according to how much it relates to the following criteria: {interp_keys[0]}  \n\n  The output must be a Python list of scores for each corresponding word in the provided list. The output must therefore have the same number of items as the provided list. The score is a float that ranges from -1 to 1. Positive scores suggest a high correlation to the criteria, while negative scores suggest a high opposite correlation. Scores closer to 0 suggest that the criterion is not applicable to the word. \n\n Here is an output sample: [<score for first word>, <score for second word>, ... , <score for second-last word>, <score for last word>] \n\n {trunc_word_list}"}
    # {"role": "user", "content": f"For the list of words below, please assign it a score according to how much it relates to the following criteria: {interp_keys[0]}  \n\n  The output must be a Python list of scores for each corresponding word in the provided list. The output must therefore have the same number of items as the provided list. The score is a float that ranges from -1 to 1. Positive scores suggest a high correlation to the criteria, while negative scores suggest a high opposite correlation. Scores closer to 0 suggest that the criterion is not applicable to the word. \n\n Here is an output sample: [<score for first word>, <score for second word>, ... , <score for second-last word>, <score for last word>] \n\n {axis_emb[0]}"}
  ]
)

print(completion.choices[0].message)

# log the stringified output into a txt file by appending it to the end of the file
with open("eval_output.txt", "a") as f:
  f.write(str(completion))

{
  "role": "assistant",
  "content": "[0.0, 0.8, 0.0, 1.0, 1.0, 0.0, -1.0, 0.9, -1.0, 0.6, 0.6, 1.0, -0.5, 0.5, -0.7, 0.9, 0.9, 0.8, 0.8, 0.0, 0.6, 0.9, -0.3, 0.9, 1.0, 0.0, 0.8, 0.7, 0.3, 0.7, 0.9, 0.6, -0.6, 0.0, -0.2, 0.5, 0.5, 0.8, 0.8, 0.7, -0.2, 1.0, -0.4, 0.9, 1.0, 0.8, -0.6, 0.5, 1.0, 0.5]"
}


In [71]:
# convert the output string into a list
score_list = eval(completion.choices[0].message.content)
print(score_list)

# check if the length is correct
print(len(score_list))

[0.0, 0.8, 0.0, 1.0, 1.0, 0.0, -1.0, 0.9, -1.0, 0.6, 0.6, 1.0, -0.5, 0.5, -0.7, 0.9, 0.9, 0.8, 0.8, 0.0, 0.6, 0.9, -0.3, 0.9, 1.0, 0.0, 0.8, 0.7, 0.3, 0.7, 0.9, 0.6, -0.6, 0.0, -0.2, 0.5, 0.5, 0.8, 0.8, 0.7, -0.2, 1.0, -0.4, 0.9, 1.0, 0.8, -0.6, 0.5, 1.0, 0.5]
50
